In [1]:
jupyter notebook \ 
    --NotebookApp.allow_origin='https://colab.research.google.com' \ 
    --port=8888 \ 
    --NotebookApp.port_retries=0

SyntaxError: unexpected character after line continuation character (3828991694.py, line 1)

In [1]:
from google.colab import drive, files, output
drive.mount('/content/drive')

# root dir in Drive where you keep the batches
DATA_ROOT = '/content/drive/MyDrive/cerpro/rotation'
# local working dir (faster I/O)
!rsync -a --progress "{DATA_ROOT}/batches/"  /content/rotation/batches/


ModuleNotFoundError: No module named 'google.colab'